In [22]:
# import libraries
import os
import json
import re
import sys
import csv
import pandas as pd
import numpy as np
import seaborn as seab
import pickle

print('headers loaded.. plotly plotting set..')

# Normalize function- to normalize values for review score and tip score//
def normalize(x, new_min = 0, new_max = 100):
    output = []
    old_min, old_max = min(x), max(x)

    for iter_val in x:
        val = (new_max - new_min) / (old_max - old_min) * (iter_val - old_min) + new_min
        output.append(val)

    return output

headers loaded.. plotly plotting set..


In [2]:
# Read dataframes for google colab
# import json
# from google.colab import drive

# drive.mount('/content/drive')
# reviews_path = "/content/drive/MyDrive/Course Work/SEM2/Info Storage and Retrieval/project/yelp_dataset/yelp_academic_dataset_review.csv"
# business_path = "/content/drive/MyDrive/Course Work/SEM2/Info Storage and Retrieval/project/yelp_dataset/yelp_academic_dataset_business.csv"
# user_path = "/content/drive/MyDrive/Course Work/SEM2/Info Storage and Retrieval/project/yelp_dataset/yelp_academic_dataset_user.csv"

# reviews_df = pd.read_csv(reviews_path)
# user_df = pd.read_csv(user_path)
# business_df = pd.read_csv(business_path)

In [23]:
# Read dataframe for Local VS Code
reviews_df = pd.read_csv('yelp_academic_dataset_review.csv')
# user_df = pd.read_csv('yelp_academic_dataset_user.csv')
business_df = pd.read_csv('yelp_academic_dataset_business.csv')

In [24]:
# Get Top Ten States
business_df = business_df.dropna(subset=['categories'])
unique_states = business_df['state'].unique()
state_map = dict()
for s in unique_states:
  state_map[s] = business_df[business_df['state'] == s].shape[0]
# 'CA' 'MO' 'AZ' 'PA' 'TN' 'FL' 'IN' 'LA' 'AB' 'NV' 'ID' 'DE' 'IL' 'NJ' 'NC' 'CO' 'WA' 'HI' 'UT' 'TX' 'MT' 'MI' 'SD' 'XMS' 'MA' 'VI' 'VT'
top_states = [state[0] for state in sorted(sorted(state_map.items(), key=lambda x: x[1], reverse=True), key=lambda x: x[1], reverse=True)[:10]]
print(top_states)

# create an empty dictionary to store the dataframes
hotel_state_df_map = {}
restaurent_state_df_map = {}
nightlife_state_df_map = {}

# Creating mask for Hotels & Travel
hotel_mask = business_df['categories'].str.contains('Hotels & Travel')
hotel_df = business_df[hotel_mask]

# Creating mask for Restaurents
restaurent_mask = business_df['categories'].str.contains('Restaurants')
restaurent_df = business_df[restaurent_mask]

# Creating mask for Nightlife
nightlife_mask = business_df['categories'].str.contains('Nightlife')
nightlife_df = business_df[nightlife_mask]

for state in top_states:
    df_name = f'business_df_{state}'

    hotel_state_df = hotel_df[hotel_df['state'] == state]
    restaurent_state_df = restaurent_df[restaurent_df['state'] == state]
    nightlife_state_df = nightlife_df[nightlife_df['state'] == state]

    exec(f"{df_name} = hotel_state_df")
    # add the dataframe to the dictionary with the state abbreviation as the key
    hotel_state_df_map[state] = hotel_state_df

    exec(f"{df_name} = restaurent_state_df")
    # add the dataframe to the dictionary with the state abbreviation as the key
    restaurent_state_df_map[state] = restaurent_state_df

    exec(f"{df_name} = nightlife_state_df")
    # add the dataframe to the dictionary with the state abbreviation as the key
    nightlife_state_df_map[state] = nightlife_state_df


['PA', 'FL', 'TN', 'IN', 'MO', 'LA', 'AZ', 'NJ', 'NV', 'AB']


In [ ]:
# nightlife_state_df_map['LA']

In [25]:
# Building Ratings Matrix
from scipy.sparse import coo_matrix
import numpy as np
from collections import defaultdict

class Recommendations:
  class Business:
    def __init__(self, name, address, city, state, postal_code, stars):
      self.name = name
      self.address = address
      self.city = city
      self.state = state
      self.postal_code = postal_code
      self.stars = stars
  
  def __init__(self, business_df, state_name, shorten=False):
    print(f"========Calculating For {state_name} State========")
    self.business_df = business_df
    self.rating_mat = []
    self.shorten = shorten if isinstance(shorten, bool) else False
    self.user_num_to_user_hash_dict = dict()
    self.user_hash_to_user_num_dict = dict()
    self.business_num_to_business_hash_dict = dict()
    self.business_hash_to_business_num_dict = dict()
    self.business_recommendations = []
    self.business_popularity = []
    self.calculateRatingMatrix()
    self.nonPersonalizedRecommendations()
  
  def calculateRatingMatrix(self):
    print("Calculating rating matrix...")
    business_list = list(self.business_df['business_id'])
    reviews_df_updated = reviews_df[reviews_df['business_id'].isin(business_list)]

    if (self.shorten):
          print(f"Size Before Cutting Down: {reviews_df_updated.shape[0]}")
          user_counts = reviews_df_updated.groupby('user_id').size().reset_index(name='count')

          # Sort the user_counts dataframe in descending order by count and select the top 100 user_ids
          top_users = user_counts.sort_values(by='count', ascending=False).head(100)['user_id'].tolist()

          # Filter the original dataframe to keep only the records that belong to the top 100 user_ids
          reviews_df_updated = reviews_df_updated[reviews_df_updated['user_id'].isin(top_users)]
          print(f"Size After Cutting Down: {reviews_df_updated.shape[0]}")


    unique_business_id = reviews_df_updated['business_id'].unique()
    unique_user_id = reviews_df_updated['user_id'].unique()

    j = 0
    for u in unique_user_id:
        self.user_hash_to_user_num_dict[u] = j
        self.user_num_to_user_hash_dict[j] = u
        j += 1

    j = 0
    for i in unique_business_id:
        self.business_hash_to_business_num_dict[i] = j
        self.business_num_to_business_hash_dict[j] = i
        j += 1

    # Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
    user_list = reviews_df_updated['user_id'].values
    movie_list = reviews_df_updated['business_id'].values
    for j in range(len(reviews_df_updated)):
        user_list[j] = self.user_hash_to_user_num_dict[user_list[j]]
        movie_list[j] = self.business_hash_to_business_num_dict[movie_list[j]]
    reviews_df_updated['user_id'] = user_list
    reviews_df_updated['business_id'] = movie_list

    num_user = len(reviews_df_updated['user_id'].unique())
    num_movie = len(reviews_df_updated['business_id'].unique())

    self.ratings_mat = coo_matrix((reviews_df_updated['stars'].values, (reviews_df_updated['user_id'].values, reviews_df_updated['business_id'].values)), shape=(num_user, num_movie)).astype(float).toarray()
    print(f"Size of Ratings Matrix: {self.ratings_mat.shape[0]}, {self.ratings_mat.shape[1]}")
  
  def nonPersonalizedRecommendations(self):
    print("Calculating NPR...")
    n = len(self.ratings_mat) # number of users
    m = len(self.ratings_mat[0]) # number of movies

    # Creating popularity array - size number of movies
    self.business_popularity = np.zeros((m,))
    self.business_popularity = self.ratings_mat.sum(axis=0) # claculating the popularity of each movie by summing the values in each column

    self.business_recommendations = np.zeros((n, 50), dtype=np.int32)

    for u in range(self.ratings_mat.shape[0]):
      business_unvisited = np.where(self.ratings_mat[u] == 0)[0]
      unwatched_popularity = self.business_popularity[business_unvisited]
      # Sort the unwatched movies according to popularity and fetch top 50 to recommend
      self.business_recommendations[u] = business_unvisited[np.argsort(unwatched_popularity)[::-1]][:50]

    # print("Non personalized recommendations for first User:")
    # for i in range(5):
    #   business_hash = self.getBusinessHashFromBusinessNum(self.business_recommendations[0,i])
    #   business = self.getBusinessInfo(business_hash)
    #   print(f"Rank {i+1}: Business {self.business_recommendations[0,i]} - Name: {business.name} - state: {business.state} - stars: {business.stars}  - Popularity {business_popularity[self.business_recommendations[0,i]]}")

  def getNPRForuUser(self, user_num):
    print(f"Non personalized recommendations for User {user_num}:")
    for i in range(5):
      business_hash = self.getBusinessHashFromBusinessNum(self.business_recommendations[0,i])
      business = self.getBusinessInfo(business_hash)
      print(f"Rank {i+1}: Business {self.business_recommendations[0,i]} - Name: {business.name} - state: {business.state} - stars: {business.stars}  - Popularity {self.business_popularity[self.business_recommendations[0,i]]}")


  def getUserHashFromUserNum(self, user_num):
    return self.user_num_to_user_hash_dict[user_num]

  def getUserNumFromUserHash(self, user_hash):
    return self.user_hash_to_user_num_dict[user_hash]

  def getBusinessHashFromBusinessNum(self, business_num):
    return self.business_num_to_business_hash_dict[business_num]

  def getBusinessNumFromBusinessHash(self, business_hash):
    return self.business_hash_to_business_num_dict[business_hash]
  
  def getBusinessInfo(self, business_hash):
    bus_df = self.business_df[self.business_df['business_id'] == business_hash].iloc[0]
    return self.Business(bus_df['name'], bus_df['address'], bus_df['city'], bus_df['state'], bus_df['postal_code'], bus_df['stars'])


In [9]:
# PA_Hotel_Recommendation = Recommendations(hotel_state_df_map['PA'], 'PA')
# FL_Hotel_Recommendation = Recommendations(hotel_state_df_map['FL'], 'FL')
# TN_Hotel_Recommendation = Recommendations(hotel_state_df_map['TN'], 'TN')
# IN_Hotel_Recommendation = Recommendations(hotel_state_df_map['IN'], 'IN')
# MO_Hotel_Recommendation = Recommendations(hotel_state_df_map['MO'], 'MO')
# LA_Hotel_Recommendation = Recommendations(hotel_state_df_map['LA'], 'LA')
# AZ_Hotel_Recommendation = Recommendations(hotel_state_df_map['AZ'], 'AZ')
# NJ_Hotel_Recommendation = Recommendations(hotel_state_df_map['NJ'], 'NJ')
# NV_Hotel_Recommendation = Recommendations(hotel_state_df_map['NV'], 'NV')
# AB_Hotel_Recommendation = Recommendations(hotel_state_df_map['AB'], 'AB')

Calculating rating matrix...


/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_19310/2907829949.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_updated['user_id'] = user_list
/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_19310/2907829949.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_updated['business_id'] = movie_list


Calculating NPR...


In [26]:
import bz2file as bz2

def compressed_pickle(title, data):
    with bz2.BZ2File(title + ".pbz2", "w") as f:
        pickle.dump(data, f)

def decompress_pickle(file):
    data = bz2.BZ2File(file, "rb")
    data = pickle.load(data)
    return data

In [ ]:
# compressed_pickle("PA_Hotel_Recommendation", PA_Hotel_Recommendation)
# compressed_pickle("FL_Hotel_Recommendation", FL_Hotel_Recommendation)
# compressed_pickle("TN_Hotel_Recommendation", TN_Hotel_Recommendation)
# compressed_pickle("IN_Hotel_Recommendation", IN_Hotel_Recommendation)
# compressed_pickle("MO_Hotel_Recommendation", MO_Hotel_Recommendation)
# compressed_pickle("LA_Hotel_Recommendation", LA_Hotel_Recommendation)
# compressed_pickle("AZ_Hotel_Recommendation", AZ_Hotel_Recommendation)
# compressed_pickle("NJ_Hotel_Recommendation", NJ_Hotel_Recommendation)
# compressed_pickle("NV_Hotel_Recommendation", NV_Hotel_Recommendation)
# compressed_pickle("AB_Hotel_Recommendation", AB_Hotel_Recommendation)

In [32]:
PA_Hotel_Recommendation = decompress_pickle("data/hotel/PA_Hotel_Recommendation.pbz2")
FL_Hotel_Recommendation = decompress_pickle("data/hotel/FL_Hotel_Recommendation.pbz2")
TN_Hotel_Recommendation = decompress_pickle("data/hotel/TN_Hotel_Recommendation.pbz2")
IN_Hotel_Recommendation = decompress_pickle("data/hotel/IN_Hotel_Recommendation.pbz2")
MO_Hotel_Recommendation = decompress_pickle("data/hotel/MO_Hotel_Recommendation.pbz2")
LA_Hotel_Recommendation = decompress_pickle("data/hotel/LA_Hotel_Recommendation.pbz2")
AZ_Hotel_Recommendation = decompress_pickle("data/hotel/AZ_Hotel_Recommendation.pbz2")
NJ_Hotel_Recommendation = decompress_pickle("data/hotel/NJ_Hotel_Recommendation.pbz2")
NV_Hotel_Recommendation = decompress_pickle("data/hotel/NV_Hotel_Recommendation.pbz2")

In [15]:
# PA_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['PA'], 'PA', True)
# FL_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['FL'], 'FL', True)
# TN_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['TN'], 'TN', True)
# IN_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['IN'], 'IN', True)
# MO_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['MO'], 'MO', True)
# LA_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['LA'], 'LA', True)
# AZ_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['AZ'], 'AZ', True)
# NJ_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['NJ'], 'NJ', True)
# NV_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['NV'], 'NV', True)
# AB_Restaurent_Recommendation = Recommendations(restaurent_state_df_map['AB'], 'AB', True)

========Calculating For PA State========
Calculating rating matrix...
Size Before Cutting Down: 1100250
Size After Cutting Down: 36442
Size of Ratings Matrix: 100, 8206
Calculating NPR...
========Calculating For FL State========
Calculating rating matrix...
Size Before Cutting Down: 792133
Size After Cutting Down: 32619
Size of Ratings Matrix: 100, 6373
Calculating NPR...
========Calculating For TN State========
Calculating rating matrix...
Size Before Cutting Down: 434697
Size After Cutting Down: 17425
Size of Ratings Matrix: 100, 3274
Calculating NPR...
========Calculating For IN State========
Calculating rating matrix...
Size Before Cutting Down: 335843
Size After Cutting Down: 21611
Size of Ratings Matrix: 100, 3238
Calculating NPR...
========Calculating For MO State========
Calculating rating matrix...
Size Before Cutting Down: 354605
Size After Cutting Down: 24585
Size of Ratings Matrix: 100, 3255
Calculating NPR...
========Calculating For LA State========
Calculating rating matr

In [19]:
# compressed_pickle("PA_Restaurent_Recommendation", PA_Restaurent_Recommendation)
# compressed_pickle("FL_Restaurent_Recommendation", FL_Restaurent_Recommendation)
# compressed_pickle("TN_Restaurent_Recommendation", TN_Restaurent_Recommendation)
# compressed_pickle("IN_Restaurent_Recommendation", IN_Restaurent_Recommendation)
# compressed_pickle("MO_Restaurent_Recommendation", MO_Restaurent_Recommendation)
# compressed_pickle("LA_Restaurent_Recommendation", LA_Restaurent_Recommendation)
# compressed_pickle("AZ_Restaurent_Recommendation", AZ_Restaurent_Recommendation)
# compressed_pickle("NJ_Restaurent_Recommendation", NJ_Restaurent_Recommendation)
# compressed_pickle("NV_Restaurent_Recommendation", NV_Restaurent_Recommendation)
# compressed_pickle("AB_Restaurent_Recommendation", AB_Restaurent_Recommendation)

In [57]:
PA_Restaurent_Recommendation = decompress_pickle("data/restaurent/PA_Restaurent_Recommendation.pbz2")
FL_Restaurent_Recommendation = decompress_pickle("data/restaurent/FL_Restaurent_Recommendation.pbz2")
TN_Restaurent_Recommendation = decompress_pickle("data/restaurent/TN_Restaurent_Recommendation.pbz2")
IN_Restaurent_Recommendation = decompress_pickle("data/restaurent/IN_Restaurent_Recommendation.pbz2")
MO_Restaurent_Recommendation = decompress_pickle("data/restaurent/MO_Restaurent_Recommendation.pbz2")
LA_Restaurent_Recommendation = decompress_pickle("data/restaurent/LA_Restaurent_Recommendation.pbz2")
AZ_Restaurent_Recommendation = decompress_pickle("data/restaurent/AZ_Restaurent_Recommendation.pbz2")
NJ_Restaurent_Recommendation = decompress_pickle("data/restaurent/NJ_Restaurent_Recommendation.pbz2")
NV_Restaurent_Recommendation = decompress_pickle("data/restaurent/NV_Restaurent_Recommendation.pbz2")
AB_Restaurent_Recommendation = decompress_pickle("data/restaurent/AB_Restaurent_Recommendation.pbz2")

In [6]:
PA_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['PA'], 'PA', True)
FL_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['FL'], 'FL', True)
TN_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['TN'], 'TN', True)
IN_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['IN'], 'IN', True)
MO_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['MO'], 'MO', True)
LA_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['LA'], 'LA', True)
AZ_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['AZ'], 'AZ', True)
NJ_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['NJ'], 'NJ', True)
NV_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['NV'], 'NV', True)
AB_Nightlife_Recommendation = Recommendations(nightlife_state_df_map['AB'], 'AB', True)

========Calculating For PA State========
Calculating rating matrix...
Size Before Cutting Down: 335684
Size After Cutting Down: 12832
Size of Ratings Matrix: 100, 1849
Calculating NPR...
========Calculating For FL State========
Calculating rating matrix...
Size Before Cutting Down: 257816
Size After Cutting Down: 12308
Size of Ratings Matrix: 100, 1728
Calculating NPR...
========Calculating For TN State========
Calculating rating matrix...
Size Before Cutting Down: 179480
Size After Cutting Down: 7192
Size of Ratings Matrix: 100, 912
Calculating NPR...
========Calculating For IN State========
Calculating rating matrix...
Size Before Cutting Down: 120505
Size After Cutting Down: 8892
Size of Ratings Matrix: 100, 844
Calculating NPR...
========Calculating For MO State========
Calculating rating matrix...
Size Before Cutting Down: 130275
Size After Cutting Down: 10154
Size of Ratings Matrix: 100, 1040
Calculating NPR...
========Calculating For LA State========
Calculating rating matrix...

In [7]:
# compressed_pickle("data/nightlife/PA_Nightlife_Recommendation", PA_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/FL_Nightlife_Recommendation", FL_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/TN_Nightlife_Recommendation", TN_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/IN_Nightlife_Recommendation", IN_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/MO_Nightlife_Recommendation", MO_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/LA_Nightlife_Recommendation", LA_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/AZ_Nightlife_Recommendation", AZ_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/NJ_Nightlife_Recommendation", NJ_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/NV_Nightlife_Recommendation", NV_Nightlife_Recommendation)
# compressed_pickle("data/nightlife/AB_Nightlife_Recommendation", AB_Nightlife_Recommendation)

In [27]:
PA_Nightlife_Recommendation = decompress_pickle("data/Nightlife/PA_Nightlife_Recommendation.pbz2")
FL_Nightlife_Recommendation = decompress_pickle("data/Nightlife/FL_Nightlife_Recommendation.pbz2")
TN_Nightlife_Recommendation = decompress_pickle("data/Nightlife/TN_Nightlife_Recommendation.pbz2")
IN_Nightlife_Recommendation = decompress_pickle("data/Nightlife/IN_Nightlife_Recommendation.pbz2")
MO_Nightlife_Recommendation = decompress_pickle("data/Nightlife/MO_Nightlife_Recommendation.pbz2")
LA_Nightlife_Recommendation = decompress_pickle("data/Nightlife/LA_Nightlife_Recommendation.pbz2")
AZ_Nightlife_Recommendation = decompress_pickle("data/Nightlife/AZ_Nightlife_Recommendation.pbz2")
NJ_Nightlife_Recommendation = decompress_pickle("data/Nightlife/NJ_Nightlife_Recommendation.pbz2")
NV_Nightlife_Recommendation = decompress_pickle("data/Nightlife/NV_Nightlife_Recommendation.pbz2")
AB_Nightlife_Recommendation = decompress_pickle("data/Nightlife/AB_Nightlife_Recommendation.pbz2")

In [13]:
class MF_implicit:
    def __init__(self, train_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        # self.user_test_like = []
        # for u in range(self.num_user):
        #     self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self):
        negative_movie = np.random.choice(np.arange(self.num_movie), size=(len(self.sample_user)), replace=True)
        true_negative = self.train_mat[self.sample_user, negative_movie] == 0
        negative_user = self.sample_user[true_negative]
        negative_movie = negative_movie[true_negative]
        return np.concatenate([self.sample_user, negative_user]), np.concatenate([self.sample_movie, negative_movie])

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of MF.
            """
            # print("Epoch:", ep+1)
            s_user, s_movie_i = self.negative_sampling()
            data = np.column_stack((s_user, s_movie_i))
            np.random.shuffle(data)
            for u, i  in zip(data[:, 0], data[:, 1]):
              actual_rating = self.train_mat[u, i]
              pu = self.P[u, :]
              qi = self.Q[i, :]

              predicted_rating = np.dot(pu, qi)
              error = 2*(predicted_rating-actual_rating)

              grad_Pu = error * qi + 2*self.reg * pu
              grad_Qi = error * pu + 2*self.reg * qi

              self.P[u, :] -= self.lr * grad_Pu
              self.Q[i, :] -= self.lr * grad_Qi
            self.predict()

            
    def predict(self):
        """
        Write your code here to implement the prediction function, which generates the ranked lists of movies 
        by the trained MF for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        """
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
          scores = prediction_mat[u]
          train_like = np.where(self.train_mat[u, :] > 0)[0]
          scores[train_like] = -9999
          top50_iid = np.argpartition(scores, -50)[-50:]
          top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
          recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation

In [14]:
def calculateMF(ratings_mat):
    mf_implicit = MF_implicit(ratings_mat, latent=5, lr=0.01, reg=0.0001)
    mf_implicit.train(epoch=20)
    recommendation = mf_implicit.predict()
    return recommendation

In [15]:
# PA_Hotel_MF = calculateMF(PA_Hotel_Recommendation.ratings_mat)
# compressed_pickle("PA_Hotel_MF", PA_Hotel_MF)
# PA_Restaurent_MF = calculateMF(PA_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("PA_Restaurent_MF", PA_Restaurent_MF)

# FL_Hotel_MF = calculateMF(FL_Hotel_Recommendation.ratings_mat)
# compressed_pickle("FL_Hotel_MF", FL_Hotel_MF)
# FL_Restaurent_MF = calculateMF(FL_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("FL_Restaurent_MF", FL_Restaurent_MF)

# TN_Hotel_MF = calculateMF(TN_Hotel_Recommendation.ratings_mat)
# compressed_pickle("TN_Hotel_MF", TN_Hotel_MF)
# TN_Restaurent_MF = calculateMF(TN_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("TN_Restaurent_MF", TN_Restaurent_MF)

# IN_Hotel_MF = calculateMF(IN_Hotel_Recommendation.ratings_mat)
# compressed_pickle("IN_Hotel_MF", IN_Hotel_MF)
# IN_Restaurent_MF = calculateMF(IN_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("IN_Restaurent_MF", IN_Restaurent_MF)

# MO_Hotel_MF = calculateMF(MO_Hotel_Recommendation.ratings_mat)
# compressed_pickle("MO_Hotel_MF", MO_Hotel_MF)
# MO_Restaurent_MF = calculateMF(MO_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("MO_Restaurent_MF", MO_Restaurent_MF)

# LA_Hotel_MF = calculateMF(LA_Hotel_Recommendation.ratings_mat)
# compressed_pickle("LA_Hotel_MF", LA_Hotel_MF)
# LA_Restaurent_MF = calculateMF(LA_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("LA_Restaurent_MF", LA_Restaurent_MF)

# AZ_Hotel_MF = calculateMF(AZ_Hotel_Recommendation.ratings_mat)
# compressed_pickle("AZ_Hotel_MF", AZ_Hotel_MF)
# AZ_Restaurent_MF = calculateMF(AZ_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("AZ_Restaurent_MF", AZ_Restaurent_MF)

# NJ_Hotel_MF = calculateMF(NJ_Hotel_Recommendation.ratings_mat)
# compressed_pickle("NJ_Hotel_MF", NJ_Hotel_MF)
# NJ_Restaurent_MF = calculateMF(NJ_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("NJ_Restaurent_MF", NJ_Restaurent_MF)

# NV_Hotel_MF = calculateMF(NV_Hotel_Recommendation.ratings_mat)
# compressed_pickle("NV_Hotel_MF", NV_Hotel_MF)
# NV_Restaurent_MF = calculateMF(NV_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("NV_Restaurent_MF", NV_Restaurent_MF)

# AB_Hotel_MF = calculateMF(AB_Hotel_Recommendation.ratings_mat)
# compressed_pickle("AB_Hotel_MF", AB_Hotel_MF)
# AB_Restaurent_MF = calculateMF(AB_Restaurent_Recommendation.ratings_mat)
# compressed_pickle("AB_Restaurent_MF", AB_Restaurent_MF)

# PA_Nightlife_MF = calculateMF(PA_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/PA_Nightlife_MF", PA_Nightlife_MF)

# FL_Nightlife_MF = calculateMF(FL_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/FL_Nightlife_MF", FL_Nightlife_MF)

# TN_Nightlife_MF = calculateMF(TN_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/TN_Nightlife_MF", TN_Nightlife_MF)

# IN_Nightlife_MF = calculateMF(IN_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/IN_Nightlife_MF", IN_Nightlife_MF)

# MO_Nightlife_MF = calculateMF(MO_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/MO_Nightlife_MF", MO_Nightlife_MF)

# LA_Nightlife_MF = calculateMF(LA_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/LA_Nightlife_MF", LA_Nightlife_MF)

# AZ_Nightlife_MF = calculateMF(AZ_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/AZ_Nightlife_MF", AZ_Nightlife_MF)

# NJ_Nightlife_MF = calculateMF(NJ_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/NJ_Nightlife_MF", NJ_Nightlife_MF)

# NV_Nightlife_MF = calculateMF(NV_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/NV_Nightlife_MF", NV_Nightlife_MF)

# AB_Nightlife_MF = calculateMF(AB_Nightlife_Recommendation.ratings_mat)
# compressed_pickle("data/nightlife/AB_Nightlife_MF", AB_Nightlife_MF)


/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_52000/3290410394.py:51: RuntimeWarning: overflow encountered in multiply
  grad_Qi = error * pu + 2*self.reg * qi
/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_52000/3290410394.py:50: RuntimeWarning: overflow encountered in multiply
  grad_Pu = error * qi + 2*self.reg * pu
/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_52000/3290410394.py:54: RuntimeWarning: invalid value encountered in subtract
  self.Q[i, :] -= self.lr * grad_Qi
/var/folders/9r/y5cvt00j367c_9w3d96w_y800000gn/T/ipykernel_52000/3290410394.py:53: RuntimeWarning: invalid value encountered in subtract
  self.P[u, :] -= self.lr * grad_Pu


In [ ]:
# Implementing Autoencoder for recommendation(basic, top-10 recommendation for each user)
import numpy as np
import tensorflow as tf
from scipy.sparse import csr_matrix

def vanilla_autoencoder():
  ratings_mat_sparse = csr_matrix(ratings_mat)

  # load data
  #ratings = np.loadtxt('ratings.csv', delimiter=',')
  n_users, n_items = ratings_mat_sparse.shape

  # split data into training and validation sets
  split = int(0.8 * n_users)
  train_ratings = ratings_mat_sparse[:split]
  val_ratings = ratings_mat_sparse[split:]

  # define the autoencoder model
  input_layer = tf.keras.layers.Input(shape=(n_items,))
  encoded_layer = tf.keras.layers.Dense(16, activation='relu')(input_layer)
  decoded_layer = tf.keras.layers.Dense(n_items, activation='sigmoid')(encoded_layer)
  autoencoder = tf.keras.models.Model(input_layer, decoded_layer)

  # compile the model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

  # train the model
  autoencoder.fit(train_ratings.toarray(), train_ratings.toarray(), epochs=20, batch_size=32, validation_data=(val_ratings, val_ratings))

  # predict ratings for all users and items
  predicted_ratings = autoencoder.predict(ratings_mat_sparse)

  # print top 10 recommended items for each user
  for i in range(n_users):
      top_items = np.argsort(predicted_ratings[i])[::-1][:10]
      print(f"User {i+1}: {top_items}")


In [ ]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from scipy.sparse import csr_matrix

def autoencoder_cf():
  ratings_mat_sparse = csr_matrix(new_rat_mat)

  # Split the data into training and testing sets
  train_size = int(0.8 * ratings_mat_sparse.shape[0])
  train_ratings = ratings_mat_sparse[:train_size, :]
  test_ratings = ratings_mat_sparse[train_size:, :]

  # Build the autoencoder model
  input_layer = Input(shape=(ratings_mat_sparse.shape[1],))
  encoded = Dense(16, activation='relu')(input_layer)
  decoded = Dense(ratings_mat_sparse.shape[1], activation='sigmoid')(encoded)
  autoencoder = Model(input_layer, decoded)

  # Compile the model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

  # Train the model
  autoencoder.fit(train_ratings.toarray(), train_ratings.toarray(), epochs=20, batch_size=32, shuffle=True, validation_data=(test_ratings.toarray(), test_ratings.toarray()))

  # Extract the hidden layer output for all the users and items
  hidden_layer = Model(input_layer, encoded)
  batch_size = 32

  def embeddings_generator(ratings_mat_sparse, batch_size):
      num_users = ratings_mat_sparse.shape[0]
      indices = np.arange(num_users)
      for start_idx in range(0, num_users, batch_size):
          end_idx = min(start_idx + batch_size, num_users)
          batch_indices = indices[start_idx:end_idx]
          batch_ratings = ratings_mat_sparse[batch_indices]
          batch_embeddings = hidden_layer.predict(batch_ratings)
          yield batch_embeddings

  user_embeddings = np.concatenate(list(embeddings_generator(ratings_mat_sparse, batch_size)), axis=0)

  # Compute the similarity between users and items
  user_similarity = np.dot(user_embeddings, user_embeddings.T)
  item_similarity = np.dot(user_embeddings.T, user_embeddings)

  # Use the similarity scores to predict the ratings of the items for each user
  user_ratings = np.dot(user_similarity, ratings_mat_sparse) / np.sum(np.abs(user_similarity), axis=1)
  item_ratings = np.dot(item_similarity, ratings_mat_sparse.T) / np.sum(np.abs(item_similarity), axis=1)

In [ ]:
# Get predicted ratings for user 
def get_user_recommendation(user_id):
  user_0_pred_ratings = user_ratings[user_id]
  N = 10
  # Sort predicted ratings in descending order
  sorted_item_indices = np.argsort(user_0_pred_ratings[0].toarray().flatten())[::-1][:N]
  print("Recommended items for user 0:", sorted_item_indices)

In [106]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
from math import sqrt


class AE_CF:
  def __init__(self, ratings_mat):
    self.ratings_mat_sparse = ratings_mat
    self.train_size = int(0.8 * self.ratings_mat_sparse.shape[0])
    self.train_ratings = self.ratings_mat_sparse[:self.train_size, :]
    self.test_ratings = self.ratings_mat_sparse[self.train_size:, :]
    self.user_ratings = []
    #self.item_ratings = []
    self.train_cf()
    self.rmse = self.calculate_rmse()

  def embeddings_generator(self, hidden_layer, batch_size):
    num_users = self.ratings_mat_sparse.shape[0]
    indices = np.arange(num_users)
    for start_idx in range(0, num_users, batch_size):
      end_idx = min(start_idx + batch_size, num_users)
      batch_indices = indices[start_idx:end_idx]
      batch_ratings = self.ratings_mat_sparse[batch_indices]
      batch_embeddings = hidden_layer.predict(batch_ratings)
      yield batch_embeddings

  def train_cf(self):
    # Build the autoencoder model
    input_layer = Input(shape=(self.ratings_mat_sparse.shape[1],))
    encoded = Dense(16, activation='relu')(input_layer)
    decoded = Dense(self.ratings_mat_sparse.shape[1], activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)

    # Compile the model
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    # Train the model
    autoencoder.fit(self.train_ratings, self.train_ratings, epochs=200, batch_size=32, shuffle=True, validation_data=(self.test_ratings, self.test_ratings))

    # Extract the hidden layer output for all the users and items
    hidden_layer = Model(input_layer, encoded)
    batch_size = 32
    user_embeddings = np.concatenate(list(self.embeddings_generator(hidden_layer, batch_size)), axis=0)

    # Compute the similarity between users and items
    user_similarity = np.dot(user_embeddings, user_embeddings.T)
    item_similarity = np.dot(user_embeddings.T, user_embeddings)

    # Use the similarity scores to predict the ratings of the items for each user
    self.user_ratings = np.dot(user_similarity, self.ratings_mat_sparse) / np.sum(np.abs(user_similarity), axis=1)[:, np.newaxis]
    #self.item_ratings = np.dot(item_similarity, self.ratings_mat_sparse.T) / np.sum(np.abs(item_similarity), axis=1)[:, np.newaxis]
  
  def calculate_rmse(self):
      mask = self.test_ratings != 0
      masked_predicted_ratings = self.user_ratings[self.train_size:, :][mask]
      masked_actual_ratings = self.test_ratings[mask]
      rmse = sqrt(mean_squared_error(masked_actual_ratings, masked_predicted_ratings))
      return rmse

    # Get predicted ratings for user 
  def get_user_recommendation(self, user_id):
      top_n = 12
      # Get the predicted ratings for the user
      user_predicted_ratings = self.user_ratings[user_id]
      # Get the indices of the items sorted by predicted ratings
      sorted_item_indices = np.argsort(user_predicted_ratings)
      # Get the indices of the top-n items
      top_n_item_indices = sorted_item_indices[-top_n:]
      return top_n_item_indices


In [107]:
def calculateAECF(ratings_mat):
    ratings_mat = ratings_mat[:100,:500]
    return AE_CF(ratings_mat)

In [ ]:
aecf_recommendations = decompress_pickle("LA_Hotel_AECF.pbz2")
recommendations_class = LA_Hotel_Recommendation

In [105]:
#np.set_printoptions(threshold=np.inf)
for x in LA_Restaurent_Recommendation.ratings_mat[0]:
    if x > 5:
        print("true")
# with open("abc.txt", "a") as f:
#   print(LA_Restaurent_Recommendation.ratings_mat[0], file=f)

true
true
true
true
true
true
true
true


In [ ]:
LA_Restaurent_AECF = calculateAECF(LA_Restaurent_Recommendation.ratings_mat)
print(LA_Restaurent_AECF.rmse)

In [103]:
for x in LA_Restaurent_AECF.test_ratings[0]:
    if x > 5:
        print("true")

true
true
true


In [79]:
business_ids = aecf_recommendations.user_ratings
#print(business_ids[0].toarray().flatten())
print(business_ids.shape)
# recommendations_class.ratings_mat.shape

# business_list = []
# for i in range(12):
#     business_hash = recommendations_class.getBusinessHashFromBusinessNum(
#         business_ids[i])
#     print(len(recommendations_class.business_num_to_business_hash_dict))
#     print(business_hash)
#     business = recommendations_class.getBusinessInfo(business_hash)
#     business_list.append(business)

(100, 100)


In [ ]:
PA_Restaurent_AECF = calculateAECF(PA_Restaurent_Recommendation.ratings_mat)
FL_Restaurent_AECF = calculateAECF(FL_Restaurent_Recommendation.ratings_mat)
TN_Restaurent_AECF = calculateAECF(TN_Restaurent_Recommendation.ratings_mat)
IN_Restaurent_AECF = calculateAECF(IN_Restaurent_Recommendation.ratings_mat)
MO_Restaurent_AECF = calculateAECF(MO_Restaurent_Recommendation.ratings_mat)
LA_Restaurent_AECF = calculateAECF(LA_Restaurent_Recommendation.ratings_mat)
AZ_Restaurent_AECF = calculateAECF(AZ_Restaurent_Recommendation.ratings_mat)
NJ_Restaurent_AECF = calculateAECF(NJ_Restaurent_Recommendation.ratings_mat)
NV_Restaurent_AECF = calculateAECF(NV_Restaurent_Recommendation.ratings_mat)
AB_Restaurent_AECF = calculateAECF(AB_Restaurent_Recommendation.ratings_mat)

In [19]:
compressed_pickle("data/reataurent/PA_Restaurent_AECF", PA_Restaurent_AECF)
compressed_pickle("data/reataurent/FL_Restaurent_AECF", FL_Restaurent_AECF)
compressed_pickle("data/reataurent/TN_Restaurent_AECF", TN_Restaurent_AECF)
compressed_pickle("data/reataurent/IN_Restaurent_AECF", IN_Restaurent_AECF)
compressed_pickle("data/reataurent/MO_Restaurent_AECF", MO_Restaurent_AECF)
compressed_pickle("data/reataurent/LA_Restaurent_AECF", LA_Restaurent_AECF)
compressed_pickle("data/reataurent/AZ_Restaurent_AECF", AZ_Restaurent_AECF)
compressed_pickle("data/reataurent/NJ_Restaurent_AECF", NJ_Restaurent_AECF)
compressed_pickle("data/reataurent/NV_Restaurent_AECF", NV_Restaurent_AECF)
compressed_pickle("data/reataurent/AB_Restaurent_AECF", AB_Restaurent_AECF)

In [ ]:
PA_Hotel_AECF = calculateAECF(PA_Hotel_Recommendation.ratings_mat)
FL_Hotel_AECF = calculateAECF(FL_Hotel_Recommendation.ratings_mat)
TN_Hotel_AECF = calculateAECF(TN_Hotel_Recommendation.ratings_mat)
IN_Hotel_AECF = calculateAECF(IN_Hotel_Recommendation.ratings_mat)
MO_Hotel_AECF = calculateAECF(MO_Hotel_Recommendation.ratings_mat)
LA_Hotel_AECF = calculateAECF(LA_Hotel_Recommendation.ratings_mat)
AZ_Hotel_AECF = calculateAECF(AZ_Hotel_Recommendation.ratings_mat)
NJ_Hotel_AECF = calculateAECF(NJ_Hotel_Recommendation.ratings_mat)
NV_Hotel_AECF = calculateAECF(NV_Hotel_Recommendation.ratings_mat)
AB_Hotel_AECF = calculateAECF(AB_Hotel_Recommendation.ratings_mat)

In [21]:
compressed_pickle("data/hotel/PA_Hotel_AECF", PA_Hotel_AECF)
compressed_pickle("data/hotel/FL_Hotel_AECF", FL_Hotel_AECF)
compressed_pickle("data/hotel/TN_Hotel_AECF", TN_Hotel_AECF)
compressed_pickle("data/hotel/IN_Hotel_AECF", IN_Hotel_AECF)
compressed_pickle("data/hotel/MO_Hotel_AECF", MO_Hotel_AECF)
compressed_pickle("data/hotel/LA_Hotel_AECF", LA_Hotel_AECF)
compressed_pickle("data/hotel/AZ_Hotel_AECF", AZ_Hotel_AECF)
compressed_pickle("data/hotel/NJ_Hotel_AECF", NJ_Hotel_AECF)
compressed_pickle("data/hotel/NV_Hotel_AECF", NV_Hotel_AECF)
compressed_pickle("data/hotel/AB_Hotel_AECF", AB_Hotel_AECF)

In [ ]:
PA_Nightlife_AECF = calculateAECF(PA_Nightlife_Recommendation.ratings_mat)
FL_Nightlife_AECF = calculateAECF(FL_Nightlife_Recommendation.ratings_mat)
TN_Nightlife_AECF = calculateAECF(TN_Nightlife_Recommendation.ratings_mat)
IN_Nightlife_AECF = calculateAECF(IN_Nightlife_Recommendation.ratings_mat)
MO_Nightlife_AECF = calculateAECF(MO_Nightlife_Recommendation.ratings_mat)
LA_Nightlife_AECF = calculateAECF(LA_Nightlife_Recommendation.ratings_mat)
AZ_Nightlife_AECF = calculateAECF(AZ_Nightlife_Recommendation.ratings_mat)
NJ_Nightlife_AECF = calculateAECF(NJ_Nightlife_Recommendation.ratings_mat)
NV_Nightlife_AECF = calculateAECF(NV_Nightlife_Recommendation.ratings_mat)
AB_Nightlife_AECF = calculateAECF(AB_Nightlife_Recommendation.ratings_mat)

In [31]:
compressed_pickle("data/Nightlife/PA_Nightlife_AECF", PA_Nightlife_AECF)
compressed_pickle("data/Nightlife/FL_Nightlife_AECF", FL_Nightlife_AECF)
compressed_pickle("data/Nightlife/TN_Nightlife_AECF", TN_Nightlife_AECF)
compressed_pickle("data/Nightlife/IN_Nightlife_AECF", IN_Nightlife_AECF)
compressed_pickle("data/Nightlife/MO_Nightlife_AECF", MO_Nightlife_AECF)
compressed_pickle("data/Nightlife/LA_Nightlife_AECF", LA_Nightlife_AECF)
compressed_pickle("data/Nightlife/AZ_Nightlife_AECF", AZ_Nightlife_AECF)
compressed_pickle("data/Nightlife/NJ_Nightlife_AECF", NJ_Nightlife_AECF)
compressed_pickle("data/Nightlife/NV_Nightlife_AECF", NV_Nightlife_AECF)
compressed_pickle("data/Nightlife/AB_Nightlife_AECF", AB_Nightlife_AECF)